# **Deep Learning Course Project**
### Course Intructor: Prof: Fabrizio Silvestri 
## **Topic: Image Super Resolution**
### Project Members:
### Saud Hussain (1990559)
### Faisal Gul (1950572)
### Rida Amjad (1684582)

## We have choosen the task of Image Super Resolution in this Deep Learning Course. 
## We have used the BSDS500 Dataset for the training and validation purposes.
## We have used Set5 Dataset for the testing purposes.
## We have implemented 4 different interpolation methods to convert images into low resolutions and then rescontruct them. 
## In addition to this for each interpolation method we are scaling the images by 2x and 4x factors.

## MOUNTING THE CONTENT FROM DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## IMPORTING ALL THE NECESSARY LIBRARIES

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
import matplotlib.pyplot as plt
import keras

## LOADING THE TRAINING AND VALIDATION DATA.


## We have used BSDS500 Dataset for the training and validation purpose.

## The dataset BSDS500 has 200 images for training, 100 for validation.

## We also normalize the images during this phase.

In [ ]:
path="drive/MyDrive/ISRDataset/BSDS500/data/"

In [ ]:
import os
import copy
from tqdm import tqdm
folder='images'

train=[]
val=[]
test=[]

for fold in os.listdir(path+folder):
  arr=[]
  
  with tqdm(total=len(os.listdir(path+folder+"/"+fold))) as pbar:
    for img in os.listdir(path+folder+"/"+fold):
      im=cv2.imread(path+folder+"/"+fold+"/"+img,0)
      try:
        im=cv2.resize(im,(80,80))
        im=im/255.0
        arr.append(im)
        pbar.update(1)
      except Exception as e:
        print("EXC: ", e)
        pass
    if fold=='train':
      train=np.array(arr)
    elif fold=='val':
      val=np.array(arr)
    elif fold=='Set5':
      test=np.array(arr)
    

train.shape

In [ ]:
#train=np.expand_dims(train,axis=3)
#train=np.squeeze(train,axis=3)
train.shape
train=np.expand_dims(train,axis=3)
val=np.expand_dims(val,axis=3)
train.shape

## Through our this function we are lowering the resolution of our images using interpolation methods of the cv2 Library.
## We resize images into 80x80 size.

## We use the scalingfactor of 2x

#### This function is inspired from the following code with modifications made by us (https://towardsdatascience.com/image-super-resolution-using-convolution-neural-networks-and-auto-encoders-28c9eceadf90)

In [ ]:
def low_resol_img_2x_inter_area(img, scaling = 2):
  w = int(img.shape[1] //2)
  h = int(img.shape[0] //2)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_AREA )
  
  
  w = int(lowresolution_img.shape[1] * 2)
  h = int(lowresolution_img.shape[0] * 2)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_AREA)
  return lowres_img

In [ ]:
train.shape
val.shape


## The low_train and low_valid contain the low resolution images. 

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_2x_inter_area(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   


low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_2x_inter_area(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

## We use the autoencoder model to train on our training images and validate on our validation images.

### This autoencoder model is inspired from the following code with little modification (https://towardsdatascience.com/image-super-resolution-using-convolution-neural-networks-and-auto-encoders-28c9eceadf90)

In [ ]:
Input_img = Input(shape=(80, 80,1))  
    
#encoding architecture
x1 = Conv2D(64, (3, 3),kernel_initializer="glorot_uniform", activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(Input_img)
x2 = Conv2D(64, (3, 3), kernel_initializer="glorot_uniform",activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x1)
x3 = MaxPool2D(padding='same')(x2)
x4 = Conv2D(128, (3, 3), kernel_initializer="glorot_uniform",activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x3)
x5 = Conv2D(128, (3, 3), kernel_initializer="glorot_uniform",activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x4)
x6 = MaxPool2D(padding='same')(x5)
encoded = Conv2D(256, (3, 3), kernel_initializer="glorot_uniform", activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x6)
# decoding architecture
x7 = UpSampling2D()(encoded)
x8 = Conv2D(128, (3, 3), kernel_initializer="glorot_uniform", activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x7)
x9 = Conv2D(128, (3, 3), kernel_initializer="glorot_uniform", activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x8)
x10 = Add()([x5, x9])
x11 = UpSampling2D()(x10)
x12 = Conv2D(64, (3, 3),kernel_initializer="glorot_uniform",  activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x11)
x13 = Conv2D(64, (3, 3), kernel_initializer="glorot_uniform", activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x12)
x14 = Add()([x2, x13])

decoded = Conv2D(3, (3, 3),kernel_initializer="glorot_uniform",  padding='same',activation='relu', kernel_regularizer=regularizers.l1(10e-10))(x14)
autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
loss_score = autoencoder.evaluate(low_valid, val)


## The validation loss with interpolation method (Inter Area) and Scaling Factor (2x) is 0.005

In [ ]:
print('validation Loss', loss_score) 

## The validation Accuracy with interpolation method (Inter Area) and Scaling Factor (2x) is 99.47

In [ ]:
print('validation accuracy', (1-loss_score)*100)

## Plotting the training and validation loss

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()



## Loading the test dataset.

## We use the SET5 Dataset for testing purpose.

## Set5 dataset consists of 5 images.

In [ ]:
import os
import copy
from tqdm import tqdm
folder='Set5/'

test=[]
arr=[]
names=[]
with tqdm(total=len(os.listdir(path+"images/"+folder))) as pbar:
  for img in os.listdir(path+"images/"+folder):
    names.append(img)
    im=cv2.imread(path+"images/"+folder+"/"+img,0)
    try:
      im=cv2.resize(im,(80,80))
      im=im/255.0
      arr.append(im)
      pbar.update(1)
    except Exception as e:
      print("EXC: ", e)
      pass
    test=np.array(arr)
    pbar.update(1)

test.shape

In [ ]:
test=np.expand_dims(test,axis=3)

## Passing the test images through low resolutin function to lower the testing images resolution

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_2x_inter_area(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)


## Predicting the outputs (low resolution-->high resolution conversion)

In [ ]:
preds=autoencoder.predict(low_test)

## We calculate the Mean Squared Error of the High Resolution and Our Reconstructed Images.
## AND
## Mean Squared Error of the High Resolution Images and Degraded Images.

## If the MSE decreases, this means our model is working well.

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

In [ ]:
preds.shape

## Visualizing the Degraded, Reconstructed and Original High Resolution Images.

In [ ]:
plt.imshow(low_test[2].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[2])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[2].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## We define function to calculate the PNSR 

In [ ]:
def PSNR(y_true,y_pred):
    mse=tf.reduce_mean( (y_true - y_pred) ** 2)
    return 20 * log10(1 / (mse ** 0.5))

def log10(x):
    numerator = tf.math.log(x)
    denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

## We calculate the PNSR of the Degraded and High Resolution and Compare it with the Reconstructed and High Resolution Images.

## If the PNSR of Reconstructed Images if greater then the Degraded, this means our model is providing good results.

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")

## Now we define a new function and keeping the interpolation function same as Inter Area but change the scaling factor to 4x


###### NOTE: Reload the training and validation, testing data again and expand their dimensions and then run the following cells. 

In [ ]:

def low_resol_img_4x_inter_area(img, scaling = 4):
  w = int(img.shape[1] //4)
  h = int(img.shape[0] //4)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_AREA )
  
  
  w = int(lowresolution_img.shape[1] * 4)
  h = int(lowresolution_img.shape[0] * 4)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_AREA)
  return lowres_img

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_4x_inter_area(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   


low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_4x_inter_area(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_4x_inter_area(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)

In [ ]:
preds=autoencoder.predict(low_test)

## MSE with interpolation Inter Area and Scaling 4x 

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

## Visualize the results

In [ ]:
plt.imshow(low_test[1].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[1])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[1].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## PNSR after interpolation is Inter Area and Scaling is 4x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")

## Now we define a new function and change the **interpolation function to Inter Nearest** and change the **scaling factor to 2x**
##### NOTE: Reload the training and validation, testing data again and expand their dimensions and then run the following cells.

In [ ]:

def low_resol_img_2x_inter_nearest(img, scaling = 2):
  w = int(img.shape[1] //2)
  h = int(img.shape[0] //2)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_NEAREST )
  
  
  w = int(lowresolution_img.shape[1] * 2)
  h = int(lowresolution_img.shape[0] * 2)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_NEAREST)
  return lowres_img

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_2x_inter_nearest(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   

low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_2x_inter_nearest(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_2x_inter_nearest(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)

In [ ]:
preds=autoencoder.predict(low_test)

## MSE with interpolation Inter Nearest and Scaling 2x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

## Visualize the results

In [ ]:
plt.imshow(low_test[2].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[2])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[2].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## PNSR after interpolation is Inter Nearest and Scaling is 2x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")

## Now we define a new function and keeping the **interpolation function same as Inter Nearest** but change the **scaling factor to 4x**
##### NOTE: Reload the training and validation, testing data again and expand their dimensions and then run the following cells.

In [ ]:
def low_resol_img_4x_inter_nearest(img, scaling = 4):
  w = int(img.shape[1] //4)
  h = int(img.shape[0] //4)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_NEAREST )
  
  
  w = int(lowresolution_img.shape[1] * 4)
  h = int(lowresolution_img.shape[0] * 4)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_NEAREST)
  return lowres_img

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_4x_inter_nearest(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   

low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_4x_inter_nearest(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_4x_inter_nearest(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)

In [ ]:
preds=autoencoder.predict(low_test)

## MSE after interpolation is Inter Nearest and Scaling is 4x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

In [ ]:
plt.imshow(low_test[1].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[1])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[1].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## PNSR after interpolation is Inter Nearest and Scaling is 4x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")

## Now we define a new function and change the interpolation function to Inter Cubic and change the scaling factor to 2x
##### NOTE: Reload the training and validation, testing data again and expand their dimensions and then run the following cells.

In [ ]:
def low_resol_img_2x_inter_cubic(img, scaling = 2):
  w = int(img.shape[1] //2)
  h = int(img.shape[0] //2)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_CUBIC )
  
  w = int(lowresolution_img.shape[1] * 2)
  h = int(lowresolution_img.shape[0] * 2)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_CUBIC)
  return lowres_img

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_2x_inter_cubic(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   


low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_2x_inter_cubic(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_2x_inter_cubic(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)

In [ ]:
preds=autoencoder.predict(low_test)

## MSE with interpolation Inter Cubic and Scaling 2x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

## Visualize the results

In [ ]:
plt.imshow(low_test[4].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[4])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[4].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## PNSR with interpolation Inter Cubic and Scaling 2x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")

## Now we define a new function and keeping the interpolation function same as Inter Cubic but change the scaling factor to 4x
##### NOTE: Reload the training and validation, testing data again and expand their dimensions and then run the following cells

In [ ]:
def low_resol_img_4x_inter_cubic(img, scaling = 4):
  w = int(img.shape[1] //4)
  h = int(img.shape[0] //4)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_CUBIC )
  
  
  w = int(lowresolution_img.shape[1] * 4)
  h = int(lowresolution_img.shape[0] * 4)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_CUBIC)
  return lowres_img

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_4x_inter_cubic(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   


low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_4x_inter_cubic(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_4x_inter_cubic(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)

In [ ]:
preds=autoencoder.predict(low_test)

## MSE with interpolation Inter Cubic and Scaling 4x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

## Visualize the results

In [ ]:
plt.imshow(low_test[4].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[4])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[4].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## PNSR after interpolation is Inter Cubic and Scaling is 4x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")

## Now we define a new function and **change the interpolation function to Inter Linear and change the scaling factor to 2x**
##### NOTE: Reload the training and validation data again and expand their dimensions and then run the following cells.

In [ ]:
def low_resol_img_2x_inter_linear(img, scaling = 2):
  w = int(img.shape[1] //2)
  h = int(img.shape[0] //2)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_LINEAR )
  
  
  w = int(lowresolution_img.shape[1] * 2)
  h = int(lowresolution_img.shape[0] * 2)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_LINEAR)
  return lowres_img

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_2x_inter_linear(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   


low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_2x_inter_linear(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_2x_inter_linear(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)

In [ ]:
preds=autoencoder.predict(low_test)

## MSE with interpolation Inter Linear and Scaling 2x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

## Visualize the results

In [ ]:
plt.imshow(low_test[1].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[1])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[1].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## PNSR with interpolation Inter Linear and Scaling 2x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")

## Now we define a new function and keeping the interpolation function **same as Inter Linear but change the scaling factor to 4x**
##### NOTE: Reload the training and validation, testing data again and expand their dimensions and then run the following cells

In [ ]:

def low_resol_img_4x_inter_linear(img, scaling = 4):
  w = int(img.shape[1] //4)
  h = int(img.shape[0] //4)
  dimensions = (w, h)
  lowresolution_img = cv2.resize(img, dimensions, interpolation = cv2.INTER_LINEAR )
  
  
  w = int(lowresolution_img.shape[1] * 4)
  h = int(lowresolution_img.shape[0] * 4)
  dimensions = (w, h)
  lowres_img = cv2.resize(lowresolution_img, dimensions, interpolation =  cv2.INTER_LINEAR)
  return lowres_img

In [ ]:
low_train = []
for i in range(train.shape[0]):
  x = low_resol_img_4x_inter_linear(train[i,:,:,:])
  low_train.append(x)
low_train = np.array(low_train)   


low_valid = []
for i in range(val.shape[0]):
  x = low_resol_img_4x_inter_linear(val[i,:,:,:])
  low_valid.append(x)
low_valid = np.array(low_valid)     

low_train.shape

In [ ]:
low_train=np.expand_dims(low_train,axis=3)
low_valid=np.expand_dims(low_valid,axis=3)

In [ ]:
history = autoencoder.fit(low_train,train,
            epochs=100,
            validation_data=(low_valid, val),
            callbacks=[early_stopper]
            )

In [ ]:
low_test = []
for i in range(test.shape[0]):
  x = low_resol_img_4x_inter_linear(test[i,:,:,:])
  low_test.append(x)
low_test = np.array(low_test)   
low_test.shape

In [ ]:
low_test=np.expand_dims(low_test,axis=3)

In [ ]:
preds=autoencoder.predict(low_test)


## MSE with interpolation Inter Linear and Scaling 4x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("MSE of High Resolution and Our Reconstructed Image")
  print(names[i],mse(test[i], pred).numpy())
  print("MSE of High Resolution and Degraded Image")
  print(names[i],mse(low_test[i], test[i]).numpy())
  print("-------------")

## Visualize the results

In [ ]:
plt.imshow(low_test[2].reshape(80,80),cmap='gray')
plt.title("Degraded")
plt.show()

In [ ]:
plt.imshow(preds[2])
plt.title("Reconstructed")
plt.show()

In [ ]:
plt.imshow(test[2].reshape(80,80),cmap='gray')
plt.title("Original")
plt.show()

## PNSR after interpolation is Inter Linear and Scaling is 4x

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()
for i,pred in enumerate(preds):
  print("PNSR of Degraded and High Resolution")
  print(names[i],PSNR(low_test[i],test[i]))
  print("PNSR of Reconstructed and High Resolution")
  print(names[i],PSNR(pred,test[i]))
  print("-------------")